In [13]:
# Inspiration :D
# https://gitlab.com/ryanlogsdon/pubsub-for-python/-/blob/main/subscriber.py
# https://www.youtube.com/watch?v=xOtrCmPjal8&ab_channel=D-I-Ry
import os
from google.cloud import pubsub_v1
import sys
import time
import pandas as pd
import http.client
import numpy as np
import datetime
from io import StringIO
import functools as ft

In [14]:
! python --version

Python 3.7.4


In [15]:
# Parameters section

location = "Hamburg"

## Create connection to PubSub

In [16]:
credentials_path = 'weather-based-forecasting-v2-c4bde37656a7.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path

In [17]:
publisher_obs = pubsub_v1.PublisherClient(
    publisher_options = pubsub_v1.types.PublisherOptions(
        enable_message_ordering=True,
    )
)

publisher_forecast = pubsub_v1.PublisherClient(
    publisher_options = pubsub_v1.types.PublisherOptions(
        enable_message_ordering=True,
    )
)

topic_path_obs = 'projects/weather-based-forecasting-v2/topics/observations_topic'
topic_path_forecast = 'projects/weather-based-forecasting-v2/topics/forecasts_topic'

## Download the data

In [18]:
time.sleep(5)



conn = http.client.HTTPSConnection("visual-crossing-weather.p.rapidapi.com")

headers = {
    "X-RapidAPI-Key": "c7b48b7c4emsh8a3b688e71b9896p1be4d0jsnda2e5b27b486",
    "X-RapidAPI-Host": "visual-crossing-weather.p.rapidapi.com"
}


conn.request("GET", "/forecast?location={}&aggregateHours=1&shortColumnNames=0&unitGroup=metric&contentType=csv".format(location),
             headers=headers)

res = conn.getresponse()
data = res.read()
s = str(data, 'utf-8')
data_string = StringIO(s)
df = pd.read_csv(data_string)

In [19]:
df2 = df.copy()
df2.index = pd.to_datetime(df2["Date time"])
df2 = df2.asfreq(freq="1min")
df2 = df2.interpolate(method ='linear', limit_direction ='forward')

df2["Date time"] = df2.index

df3 = df2.iloc[:60*24*10, :]

In [20]:
df3

,Address,Date time,Latitude,Longitude,Resolved Address,Name,Wind Direction,Temperature,Wind Speed,Cloud Cover,Heat Index,Chance Precipitation (%),Precipitation,Sea Level Pressure,Snow Depth,Snow,Relative Humidity,Wind Gust,Wind Chill,Conditions
Date time,,,,,,,,,,,,,,,,,,,,
2022-12-30 20:00:00,Hamburg,2022-12-30 20:00:00,53.5537,9.99183,"Hamburg, Deutschland",Hamburg,176.800000,7.800000,21.600000,100.000000,NaN,100.000000,0.400000,999.000,0.0,0.0,79.600000,48.200000,4.400000,"Rain, Overcast"
2022-12-30 20:01:00,NaN,2022-12-30 20:01:00,53.5537,9.99183,NaN,NaN,176.971667,7.810000,21.581667,100.000000,NaN,100.000000,0.395000,999.000,0.0,0.0,79.630000,48.188333,4.415000,NaN
2022-12-30 20:02:00,NaN,2022-12-30 20:02:00,53.5537,9.99183,NaN,NaN,177.143333,7.820000,21.563333,100.000000,NaN,100.000000,0.390000,999.000,0.0,0.0,79.660000,48.176667,4.430000,NaN
2022-12-30 20:03:00,NaN,2022-12-30 20:03:00,53.5537,9.99183,NaN,NaN,177.315000,7.830000,21.545000,100.000000,NaN,100.000000,0.385000,999.000,0.0,0.0,79.690000,48.165000,4.445000,NaN
2022-12-30 20:04:00,NaN,2022-12-30 20:04:00,53.5537,9.99183,NaN,NaN,177.486667,7.840000,21.526667,100.000000,NaN,100.000000,0.380000,999.000,0.0,0.0,79.720000,48.153333,4.460000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-09 19:55:00,NaN,2023-01-09 19:55:00,53.5537,9.99183,NaN,NaN,241.558333,1.908333,21.933333,31.383333,NaN,56.708333,0.008333,1004.275,0.3,0.0,83.033333,50.250000,-3.083333,NaN
2023-01-09 19:56:00,NaN,2023-01-09 19:56:00,53.5537,9.99183,NaN,NaN,241.546667,1.906667,21.946667,31.266667,NaN,56.786667,0.006667,1004.280,0.3,0.0,83.046667,50.260000,-3.086667,NaN
2023-01-09 19:57:00,NaN,2023-01-09 19:57:00,53.5537,9.99183,NaN,NaN,241.535000,1.905000,21.960000,31.150000,NaN,56.865000,0.005000,1004.285,0.3,0.0,83.060000,50.270000,-3.090000,NaN


In [21]:
now = datetime.datetime.now()
minutes = now.minute
df3_hour = df3.iloc[minutes-1:minutes+59, :]
df3_hour.head(5)

,Address,Date time,Latitude,Longitude,Resolved Address,Name,Wind Direction,Temperature,Wind Speed,Cloud Cover,Heat Index,Chance Precipitation (%),Precipitation,Sea Level Pressure,Snow Depth,Snow,Relative Humidity,Wind Gust,Wind Chill,Conditions
Date time,,,,,,,,,,,,,,,,,,,,
2022-12-30 20:39:00,NaN,2022-12-30 20:39:00,53.5537,9.99183,NaN,NaN,183.495000,8.19,20.885000,100.0,NaN,100.0,0.205,999.0,0.0,0.0,80.77,47.745000,4.985,NaN
2022-12-30 20:40:00,NaN,2022-12-30 20:40:00,53.5537,9.99183,NaN,NaN,183.666667,8.20,20.866667,100.0,NaN,100.0,0.200,999.0,0.0,0.0,80.80,47.733333,5.000,NaN
2022-12-30 20:41:00,NaN,2022-12-30 20:41:00,53.5537,9.99183,NaN,NaN,183.838333,8.21,20.848333,100.0,NaN,100.0,0.195,999.0,0.0,0.0,80.83,47.721667,5.015,NaN
2022-12-30 20:42:00,NaN,2022-12-30 20:42:00,53.5537,9.99183,NaN,NaN,184.010000,8.22,20.830000,100.0,NaN,100.0,0.190,999.0,0.0,0.0,80.86,47.710000,5.030,NaN
2022-12-30 20:43:00,NaN,2022-12-30 20:43:00,53.5537,9.99183,NaN,NaN,184.181667,8.23,20.811667,100.0,NaN,100.0,0.185,999.0,0.0,0.0,80.89,47.698333,5.045,NaN


## Send Forecast in intervals to PubSub

In [22]:
df3_forecast = df3.iloc[minutes-1:, :]
df3_forecast.index = df3_forecast.index.astype(str)
df3_forecast.head(5)

,Address,Date time,Latitude,Longitude,Resolved Address,Name,Wind Direction,Temperature,Wind Speed,Cloud Cover,Heat Index,Chance Precipitation (%),Precipitation,Sea Level Pressure,Snow Depth,Snow,Relative Humidity,Wind Gust,Wind Chill,Conditions
Date time,,,,,,,,,,,,,,,,,,,,
2022-12-30 20:39:00,NaN,2022-12-30 20:39:00,53.5537,9.99183,NaN,NaN,183.495000,8.19,20.885000,100.0,NaN,100.0,0.205,999.0,0.0,0.0,80.77,47.745000,4.985,NaN
2022-12-30 20:40:00,NaN,2022-12-30 20:40:00,53.5537,9.99183,NaN,NaN,183.666667,8.20,20.866667,100.0,NaN,100.0,0.200,999.0,0.0,0.0,80.80,47.733333,5.000,NaN
2022-12-30 20:41:00,NaN,2022-12-30 20:41:00,53.5537,9.99183,NaN,NaN,183.838333,8.21,20.848333,100.0,NaN,100.0,0.195,999.0,0.0,0.0,80.83,47.721667,5.015,NaN
2022-12-30 20:42:00,NaN,2022-12-30 20:42:00,53.5537,9.99183,NaN,NaN,184.010000,8.22,20.830000,100.0,NaN,100.0,0.190,999.0,0.0,0.0,80.86,47.710000,5.030,NaN
2022-12-30 20:43:00,NaN,2022-12-30 20:43:00,53.5537,9.99183,NaN,NaN,184.181667,8.23,20.811667,100.0,NaN,100.0,0.185,999.0,0.0,0.0,80.89,47.698333,5.045,NaN


In [23]:
for i in range(3):
    
    df3_forecast_window = str(df3_forecast.iloc[i:i+3*24*10,:].astype(str).to_dict("index"))
    df3_forecast_window = df3_forecast_window.encode('utf-8')
        
    info = "Weather forecast for Hamburg, {}".format(df3_forecast.index[i])
    info = info.encode('utf-8')
    
    future = publisher_forecast.publish(topic_path_forecast, df3_forecast_window)
    print(f'published message id {future.result()}')
    time.sleep(60)

published message id 6684007539860929
published message id 6684008110283454
published message id 6684028289426570
